In [1]:
#!pip install transformers[torch]==4.10.0
#!pip install --upgrade tensorflow
#pip install --upgrade simplet5
# !pip install pytorch-lightning==1.4.5

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.model_selection import train_test_split
from tqdm import notebook as tqdm
#from simplet5 import SimpleT5
import pandas as pd
import re 
from tqdm import tqdm_notebook as tqdm
import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from transformers import (
    Adafactor,
    T5ForConditionalGeneration,
    MT5ForConditionalGeneration,
    #ByT5Tokenizer,
    PreTrainedTokenizer,
    T5TokenizerFast as T5Tokenizer,
    MT5TokenizerFast as MT5Tokenizer,
)
from transformers import AutoTokenizer

# from fastT5 import export_and_get_onnx_model
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelWithLMHead, AutoTokenizer
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

import os

import sys
import random
import numpy as np
import argparse
from PIL import Image
import matplotlib.pyplot as plt
from datetime import datetime
from uuid import uuid4
#os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [3]:
input_data_name = "claim_LOF_base_0.45_0.575_11.905_data_explanation_prep_4.pickle"                            #"LOF_base_0.45_0.53_removed_inlier_outlier_23.782_full.pickle"     # "LOF_base_0.46_0.54_removed_inlier_outlier_0.51_full.pickle"
data_inpit_dir = "./Data/Selection/"                              #"./Data/Selection/" "./Data/Preprocessed/"   
output_dir = "./Data/Models/"
source_column = "source_text_shorter"    #source_text_shorter           " statement_explanation_prep"  #"explanation_prep"  "statement_explanation_prep"                     #"source_text_shorter"  # "source_text_shorter" source_text
target_column =  "target_text"              #"shortExplanation_prep"                         #"target_text"

no_workers = 1

imput_data_path = data_inpit_dir + input_data_name
#input_data_name = re.sub(r'.pickle', '', input_data_name)
new_model_name = "d-t5-{}_{}".format(source_column, input_data_name) # datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

In [4]:
torch.cuda.get_device_name(0)

'A100-SXM4-40GB'

In [5]:
new_model_name

'd-t5-source_text_shorter_claim_LOF_base_0.45_0.575_11.905_data_explanation_prep_4.pickle'

In [6]:
torch.cuda.empty_cache()
pl.seed_everything(42)


class PyTorchDataModule(Dataset):
    """  PyTorch Dataset class  """

    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: PreTrainedTokenizer,
        source_max_token_len: int = 512,
        target_max_token_len: int = 512,
    ):
        """
        initiates a PyTorch Dataset Module for input data
        Args:
            data (pd.DataFrame): input pandas dataframe. Dataframe must have 2 column --> "source_text" and "target_text"
            tokenizer (PreTrainedTokenizer): a PreTrainedTokenizer (T5Tokenizer, MT5Tokenizer, or ByT5Tokenizer)
            source_max_token_len (int, optional): max token length of source text. Defaults to 512.
            target_max_token_len (int, optional): max token length of target text. Defaults to 512.
        """
        self.tokenizer = tokenizer
        self.data = data
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def __len__(self):
        """ returns length of data """
        return len(self.data)

    def __getitem__(self, index: int):
        """ returns dictionary of input tensors to feed into T5/MT5 model"""

        data_row = self.data.iloc[index]
        source_text = data_row["source_text"]

        source_text_encoding = self.tokenizer(
            source_text,
            max_length=self.source_max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt",
        )

        target_text_encoding = self.tokenizer(
            data_row["target_text"],
            max_length=self.target_max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt",
        )

        labels = target_text_encoding["input_ids"]
        labels[
            labels == 0
        ] = -100  # to make sure we have correct labels for T5 text generation

        return dict(
            source_text=source_text,
            target_text=data_row["target_text"],
            source_text_input_ids=source_text_encoding["input_ids"].flatten(),
            source_text_attention_mask=source_text_encoding["attention_mask"].flatten(),
            labels=labels.flatten(),
            labels_attention_mask=target_text_encoding["attention_mask"].flatten(),
        )


class LightningDataModule(pl.LightningDataModule):
    """ PyTorch Lightning data class """

    def __init__(
        self,
        train_df: pd.DataFrame,
        test_df: pd.DataFrame,
        tokenizer: PreTrainedTokenizer,
        batch_size: int = 4,
        source_max_token_len: int = 512,
        target_max_token_len: int = 512,
    ):
        """
        initiates a PyTorch Lightning Data Module
        Args:
            train_df (pd.DataFrame): training dataframe. Dataframe must contain 2 columns --> "source_text" & "target_text"
            test_df (pd.DataFrame): validation dataframe. Dataframe must contain 2 columns --> "source_text" & "target_text"
            tokenizer (PreTrainedTokenizer): PreTrainedTokenizer (T5Tokenizer, MT5Tokenizer, or ByT5Tokenizer)
            batch_size (int, optional): batch size. Defaults to 4.
            source_max_token_len (int, optional): max token length of source text. Defaults to 512.
            target_max_token_len (int, optional): max token length of target text. Defaults to 512.
        """
        super().__init__()

        self.train_df = train_df
        self.test_df = test_df
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def setup(self, stage=None):
        self.train_dataset = PyTorchDataModule(
            self.train_df,
            self.tokenizer,
            self.source_max_token_len,
            self.target_max_token_len,
        )
        self.test_dataset = PyTorchDataModule(
            self.test_df,
            self.tokenizer,
            self.source_max_token_len,
            self.target_max_token_len,
        )

    def train_dataloader(self):
        """ training dataloader """
        return DataLoader(
            self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers = no_workers
        )

    def test_dataloader(self):
        """ test dataloader """
        return DataLoader(
            self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers = no_workers
        )

    def val_dataloader(self):
        """ validation dataloader """
        return DataLoader(
            self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers = no_workers
        )


class LightningModel(pl.LightningModule):
    """ PyTorch Lightning Model class"""

    def __init__(self, tokenizer, model, outputdir: str = "outputs"):
        """
        initiates a PyTorch Lightning Model
        Args:
            tokenizer : T5/MT5/ByT5 tokenizer
            model : T5/MT5/ByT5 model
            outputdir (str, optional): output directory to save model checkpoints. Defaults to "outputs".
        """
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.outputdir = outputdir

    def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):
        """ forward step """
        output = self.model(
            input_ids,
            attention_mask=attention_mask,
            labels=labels,
            decoder_attention_mask=decoder_attention_mask,
        )

        return output.loss, output.logits

    def training_step(self, batch, batch_size):
        """ training step """
        input_ids = batch["source_text_input_ids"]
        attention_mask = batch["source_text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=labels_attention_mask,
            labels=labels,
        )

        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_size):
        """ validation step """
        input_ids = batch["source_text_input_ids"]
        attention_mask = batch["source_text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=labels_attention_mask,
            labels=labels,
        )

        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_size):
        """ test step """
        input_ids = batch["source_text_input_ids"]
        attention_mask = batch["source_text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=labels_attention_mask,
            labels=labels,
        )

        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        """ configure optimizers """
        return Adafactor(self.parameters(), lr=0.001, scale_parameter=False, relative_step=False)

    def training_epoch_end(self, training_step_outputs):
        """ save tokenizer and model on epoch end """
        avg_traning_loss = np.round(
            torch.mean(torch.stack([x["loss"] for x in training_step_outputs])).item(),
            4,
        )
        self.avg_traning_loss = avg_traning_loss
        path = f"{self.outputdir}/simplet5-epoch-{self.current_epoch}-train-loss-{str(avg_traning_loss)}"
        print("Average training loss for epoch {} equal to {}".format(self.current_epoch,self.avg_traning_loss))
        #self.tokenizer.save_pretrained(path)
        #self.model.save_pretrained(path)

    # def validation_epoch_end(self, validation_step_outputs):
    #     # val_loss = torch.stack([x['loss'] for x in validation_step_outputs]).mean()
    #     path = f"{self.outputdir}/T5-epoch-{self.current_epoch}"
    #     self.tokenizer.save_pretrained(path)
    #     # self.model.save_pretrained(path)


class SimpleT5:
    """ Custom SimpleT5 class """

    def __init__(self) -> None:
        """ initiates SimpleT5 class """
        pass

    def from_pretrained(self, model_type="t5", model_name="t5-base") -> None:
        """
        loads T5/MT5 Model model for training/finetuning
        Args:
            model_type (str, optional): "t5" or "mt5" . Defaults to "t5".
            model_name (str, optional): exact model architecture name, "t5-base" or "t5-large". Defaults to "t5-base".
        """
        if model_type == "t5":
            self.tokenizer = T5Tokenizer.from_pretrained(f"{model_name}")
            self.model = T5ForConditionalGeneration.from_pretrained(
                f"{model_name}", return_dict=True
            )
        elif model_type == "mt5":
            self.tokenizer = MT5Tokenizer.from_pretrained(f"{model_name}")
            self.model = MT5ForConditionalGeneration.from_pretrained(
                f"{model_name}", return_dict=True
            )
        elif model_type == "byt5":
            self.tokenizer = ByT5Tokenizer.from_pretrained(f"{model_name}")
            self.model = T5ForConditionalGeneration.from_pretrained(
                f"{model_name}", return_dict=True
            )

    def train(
        self,
        train_df: pd.DataFrame,
        eval_df: pd.DataFrame,
        source_max_token_len: int = 512,
        target_max_token_len: int = 512,
        batch_size: int = 8,
        max_epochs: int = 5,
        use_gpu: bool = True,
        outputdir: str = "outputs",
        early_stopping_patience_epochs: int = 0,  # 0 to disable early stopping feature
        precision=32,
    ):
        """
        trains T5/MT5 model on custom dataset
        Args:
            train_df (pd.DataFrame): training datarame. Dataframe must have 2 column --> "source_text" and "target_text"
            eval_df ([type], optional): validation datarame. Dataframe must have 2 column --> "source_text" and "target_text"
            source_max_token_len (int, optional): max token length of source text. Defaults to 512.
            target_max_token_len (int, optional): max token length of target text. Defaults to 512.
            batch_size (int, optional): batch size. Defaults to 8.
            max_epochs (int, optional): max number of epochs. Defaults to 5.
            use_gpu (bool, optional): if True, model uses gpu for training. Defaults to True.
            outputdir (str, optional): output directory to save model checkpoints. Defaults to "outputs".
            early_stopping_patience_epochs (int, optional): monitors val_loss on epoch end and stops training, if val_loss does not improve after the specied number of epochs. set 0 to disable early stopping. Defaults to 0 (disabled)
            precision (int, optional): sets precision training - Double precision (64), full precision (32) or half precision (16). Defaults to 32.
        """
        self.target_max_token_len = target_max_token_len
        self.data_module = LightningDataModule(
            train_df,
            eval_df,
            self.tokenizer,
            batch_size=batch_size,
            source_max_token_len=source_max_token_len,
            target_max_token_len=target_max_token_len,
        )

        self.T5Model = LightningModel(
            tokenizer = self.tokenizer, model=self.model, outputdir=outputdir
        )

        # checkpoint_callback = ModelCheckpoint(
        #     dirpath="checkpoints",
        #     filename="best-checkpoint-{epoch}-{train_loss:.2f}",
        #     save_top_k=-1,
        #     verbose=True,
        #     monitor="train_loss",
        #     mode="min",
        # )

        # logger = TensorBoardLogger("SimpleT5", name="SimpleT5-Logger")

        early_stop_callback = (
            [
                EarlyStopping(
                    monitor="val_loss",
                    min_delta=0.00,
                    patience=early_stopping_patience_epochs,
                    verbose=True,
                    mode="min",
                )
            ]
            if early_stopping_patience_epochs > 0
            else None
        )

        gpus = 1 if use_gpu else 0

        trainer = pl.Trainer(
            # logger=logger,
            callbacks=early_stop_callback,
            max_epochs=max_epochs,
            gpus=gpus,
            progress_bar_refresh_rate = 5,
            precision=precision
        )

        trainer.fit(self.T5Model, self.data_module)
        #path = f"{self.T5Model.outputdir}/simplet5-epoch-{self.T5Model.current_epoch}-train-loss-{str(self.T5Model.avg_traning_loss)}"
        path = f"{self.T5Model.outputdir}" + new_model_name
        #print(str(avg_traning_loss))
        self.tokenizer.save_pretrained(path)
        self.model.save_pretrained(path)

    def load_model(
        self, model_type: str = "t5", model_dir: str = "outputs", use_gpu: bool = False
    ):
        """
        loads a checkpoint for inferencing/prediction
        Args:
            model_type (str, optional): "t5" or "mt5". Defaults to "t5".
            model_dir (str, optional): path to model directory. Defaults to "outputs".
            use_gpu (bool, optional): if True, model uses gpu for inferencing/prediction. Defaults to True.
        """
        if model_type == "t5":
            self.model = T5ForConditionalGeneration.from_pretrained(f"{model_dir}")
            self.tokenizer = T5Tokenizer.from_pretrained(f"{model_dir}")
        elif model_type == "mt5":
            self.model = MT5ForConditionalGeneration.from_pretrained(f"{model_dir}")
            self.tokenizer = MT5Tokenizer.from_pretrained(f"{model_dir}")
        elif model_type == "byt5":
            self.model = T5ForConditionalGeneration.from_pretrained(f"{model_dir}")
            self.tokenizer = ByT5Tokenizer.from_pretrained(f"{model_dir}")

        if use_gpu:
            if torch.cuda.is_available():
                self.device = torch.device("cuda")
            else:
                raise "exception ---> no gpu found. set use_gpu=False, to use CPU"
        else:
            self.device = torch.device("cpu")

        self.model = self.model.to(self.device)

    def predict(
        self,
        source_text: str,
        max_length: int = 512,
        num_return_sequences: int = 1,
        num_beams: int = 2,
        top_k: int = 50,
        top_p: float = 0.95,
        do_sample: bool = True,
        repetition_penalty: float = 2.5,
        length_penalty: float = 1.0,
        early_stopping: bool = True,
        skip_special_tokens: bool = True,
        clean_up_tokenization_spaces: bool = True,
    ):
        """
        generates prediction for T5/MT5 model
        Args:
            source_text (str): any text for generating predictions
            max_length (int, optional): max token length of prediction. Defaults to 512.
            num_return_sequences (int, optional): number of predictions to be returned. Defaults to 1.
            num_beams (int, optional): number of beams. Defaults to 2.
            top_k (int, optional): Defaults to 50.
            top_p (float, optional): Defaults to 0.95.
            do_sample (bool, optional): Defaults to True.
            repetition_penalty (float, optional): Defaults to 2.5.
            length_penalty (float, optional): Defaults to 1.0.
            early_stopping (bool, optional): Defaults to True.
            skip_special_tokens (bool, optional): Defaults to True.
            clean_up_tokenization_spaces (bool, optional): Defaults to True.
        Returns:
            list[str]: returns predictions
        """
        input_ids = self.tokenizer.encode(
            source_text, return_tensors="pt", add_special_tokens=True
        )
        input_ids = input_ids.to(self.device)
        generated_ids = self.model.generate(
            input_ids=input_ids,
            num_beams=num_beams,
            max_length=max_length,
            repetition_penalty=repetition_penalty,
            length_penalty=length_penalty,
            early_stopping=early_stopping,
            top_p=top_p,
            top_k=top_k,
            num_return_sequences=num_return_sequences,
        )
        preds = [
            self.tokenizer.decode(
                g,
                skip_special_tokens=skip_special_tokens,
                clean_up_tokenization_spaces=clean_up_tokenization_spaces,
            )
            for g in generated_ids
        ]
        return preds

    # def convert_and_load_onnx_model(self, model_dir: str):
    #     """ returns ONNX model """
    #     self.onnx_model = export_and_get_onnx_model(model_dir)
    #     self.onnx_tokenizer = AutoTokenizer.from_pretrained(model_dir)

    # def onnx_predict(self, source_text: str):
    #     """ generates prediction from ONNX model """
    #     token = self.onnx_tokenizer(source_text, return_tensors="pt")
    #     tokens = self.onnx_model.generate(
    #         input_ids=token["input_ids"],
    #         attention_mask=token["attention_mask"],
    #         num_beams=2,
    #     )
    #     output = self.onnx_tokenizer.decode(tokens.squeeze(), skip_special_tokens=True)
    #     return output

Global seed set to 42


In [7]:
data = pd.read_pickle(imput_data_path)

In [8]:
train, dev_test = train_test_split(data, test_size = 0.2, random_state = 42)
dev, test = train_test_split(dev_test, test_size = 0.5, random_state = 42)

In [9]:
train['target_text'] = train[target_column]
train['source_text'] = "summarize: " + train[source_column]

train['target_len'] = train["target_text"].str.split().str.len()
train['source_len'] = train["source_text"].str.split().str.len()
train[['target_len','source_len']].describe()

,target_len,source_len
count,10312.000000,10312.000000
mean,85.508146,731.882758
std,41.681467,270.750197
min,2.000000,41.000000
25%,58.000000,540.000000
50%,80.000000,696.000000
75%,106.250000,879.000000
max,775.000000,2925.000000


In [10]:
len(train)

10312

In [11]:
dev['target_text'] = dev[target_column]
dev['source_text'] = "summarize: " + dev[source_column]

dev['target_len'] = dev["target_text"].str.split().str.len()
dev['source_len'] = dev["source_text"].str.split().str.len()
dev[['target_len','source_len']].describe()

,target_len,source_len
count,1289.000000,1289.000000
mean,85.415050,737.681924
std,41.921667,290.569300
min,9.000000,166.000000
25%,58.000000,533.000000
50%,80.000000,692.000000
75%,106.000000,888.000000
max,486.000000,2878.000000


In [12]:
train = train[["target_text", "source_text"]]
dev = dev[["target_text","source_text"]]

In [13]:
model = SimpleT5()

In [14]:
model.from_pretrained(model_type="t5",model_name = "t5-base") # large  "google/mt5-base"

In [15]:
import gc
#del data, model
gc.collect()
import torch
torch.cuda.empty_cache()

In [16]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [17]:
model.train(train_df = train, #45_56
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-dd659cb7-e00c-a571-5359-f0f188a08fee]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2366


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9315


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7201


In [17]:
model.train(train_df = train, #45_56
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-69415620-de6c-329d-ef8d-9c9c937f4850]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2256


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9291


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7165


In [17]:
model.train(train_df = train, #45_56
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-1cad794e-e425-8f4b-3a02-a512bba3497a]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/storage/brno3-cerit/home/vajp02/.conda/envs/myenv/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
/storage/brno3-cerit/home/vajp02/.conda/envs/myenv/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2921


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9845


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7736


In [18]:
model.train(train_df = train, #45_56
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-bceb1af3-48fa-7240-a710-a81ba0d3ebc5]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2872


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9705


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7516


In [18]:
model.train(train_df = train, #45_56
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-129c91a9-fac6-0c27-6de4-14ecc17b6501]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2267


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.922


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7125


In [18]:
model.train(train_df = train, #45_57
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-129c91a9-fac6-0c27-6de4-14ecc17b6501]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2506


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9477


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.736


In [18]:
model.train(train_df = train, #45_56
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-c0eb6ae4-7a40-b947-996f-5811490f6f65]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.235


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9299


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7274


In [18]:
model.train(train_df = train, #45_56
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-129c91a9-fac6-0c27-6de4-14ecc17b6501]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2565


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9485


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7415


In [18]:
model.train(train_df = train, #45_55
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-129c91a9-fac6-0c27-6de4-14ecc17b6501]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2759


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9622


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7635


In [36]:
model.train(train_df = train, #45_54
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-129c91a9-fac6-0c27-6de4-14ecc17b6501]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.3017


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.992


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7793


In [18]:
model.train(train_df = train, # 57
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-129c91a9-fac6-0c27-6de4-14ecc17b6501]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2577


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9523


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7442


In [18]:
model.train(train_df = train,
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-129c91a9-fac6-0c27-6de4-14ecc17b6501]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.3315


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 2.0217


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.8072


In [18]:
model.train(train_df = train,
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-129c91a9-fac6-0c27-6de4-14ecc17b6501]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.3992


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 2.0866


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.8762


In [18]:
model.train(train_df = train,
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-129c91a9-fac6-0c27-6de4-14ecc17b6501]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.3697


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 2.0543


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.8423


In [18]:
model.train(train_df = train,
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-c85b1185-eb3f-7f99-412a-2486fe49bf7c]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.3461


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 2.0335


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.8241


In [18]:
model.train(train_df = train,
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 150,  #100 max_shortexplanation_tokens
            batch_size = 8, max_epochs = 3, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-c85b1185-eb3f-7f99-412a-2486fe49bf7c]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Average training loss for epoch 0 equal to 2.2937


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 1 equal to 1.9792


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 2 equal to 1.7665


In [17]:
model.train(train_df = train,
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 100,  #100 max_shortexplanation_tokens
            batch_size = 9, max_epochs = 3, # 18
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-bceb1af3-48fa-7240-a710-a81ba0d3ebc5]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Metric val_loss improved by 0.091 >= min_delta = 0.0. New best score: 1.882


Average training loss for epoch 1 equal to 1.7684


Validating: 0it [00:00, ?it/s]

Monitored metric val_loss did not improve in the last 1 records. Best score: 1.882. Signaling Trainer to stop.


Average training loss for epoch 2 equal to 1.5647


In [18]:
model.train(train_df = train,
            eval_df = dev, 
            source_max_token_len = 1200,  #2000 max_explanation_tokens
            target_max_token_len = 100,  #100 max_shortexplanation_tokens
            batch_size = 9, max_epochs = 6, # 18
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 0) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-a527da35-0f6b-a3bf-de28-5bb4f9ba4001]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Average training loss for epoch 3 equal to 1.444


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 4 equal to 1.2971


Validating: 0it [00:00, ?it/s]

Average training loss for epoch 5 equal to 1.1629
